# Testing roads coordinates array concatenation

In [1]:
def printList(list):
    for i, coord in enumerate(list):
        print("{}. https://www.openstreetmap.org/?mlat={}&mlon={}&zoom=19".format(i, coord[1], coord[0]))

# Testing concatenation algorithm

In [12]:
data = [
    {
     "geometry":{
        "type":"MultiLineString",
        "coordinates":[
           [
              [
                 14.4070888,
                 50.124456
              ],
              [
                 14.40784007,
                 50.12440019
              ],
              [
                 14.40792462,
                 50.1244215
              ]
           ],
           [
              [
                 14.40808668,
                 50.12340308
              ],
              [
                 14.4076876,
                 50.12366512
              ]
           ]
        ]
     },
     "type":"Feature",
     "properties":{
        "source":"openstreetmap.org",
        "kind":"path",
        "kind_detail":"path",
        "sort_rank":354,
        "landuse_kind":"grass",
        "min_zoom":13.0,
        "id":173868646
     }
    },
    {
     "geometry":{
        "type":"MultiLineString",
        "coordinates":[
           [
              [
                 14.4076876,
                 50.12366512
              ],
              [
                 14.40733297,
                 50.12389797
              ],
              [
                 14.40702799,
                 50.12400101
              ],
              [
                 14.40629802,
                 50.12403101
              ],
              [
                 14.40585973,
                 50.12400078
              ],
              [
                 14.40570397,
                 50.12398097
              ],
              [
                 14.40525696,
                 50.12366898
              ],
              [
                 14.40428831,
                 50.12338498
              ],
              [
                 14.40421716,
                 50.12358011
              ],
              [
                 14.40517701,
                 50.12418398
              ],
              [
                 14.40524987,
                 50.12423426
              ],
              [
                 14.40564396,
                 50.12442397
              ],
              [
                 14.40577305,
                 50.12444707
              ],
              [
                 14.40613309,
                 50.12452701
              ],
              [
                 14.4070888,
                 50.124456
              ]
           ],
           [
              [
                 14.40792462,
                 50.1244215
              ],
              [
                 14.40821179,
                 50.12449389
              ],
              [
                 14.40854983,
                 50.1245952
              ],
              [
                 14.40876406,
                 50.12474902
              ]
           ],
           [
              [
                 14.40827791,
                 50.12327751
              ],
              [
                 14.40808668,
                 50.12340308
              ]
           ]
        ]
     },
     "type":"Feature",
     "properties":{
        "source":"openstreetmap.org",
        "kind":"path",
        "kind_detail":"path",
        "sort_rank":354,
        "landuse_kind":"forest",
        "min_zoom":13.0,
        "id":173868646
     }
    },
    {
     "geometry":{
        "type":"LineString",
        "coordinates":[
           [
              14.40876406,
              50.12474902
           ],
           [
              14.40883298,
              50.1247985
           ]
        ]
     },
     "type":"Feature",
     "properties":{
        "source":"openstreetmap.org",
        "kind":"path",
        "sort_rank":354,
        "kind_detail":"path",
        "min_zoom":13.0,
        "id":173868646
     }
    }
]

In [13]:
# Reading GeoJSONS, storing roads and combining roads with the same id into one road
# All roads now have MultiLineString geometry with coordinate tuples

def prepare_geometry(geometry: dict) -> dict:
    """
    Changes all geometries to MultiLineString type and each coordinate into a tuple.
    """
    coords = list()
    
    if(geometry['type'] == 'LineString'):
        coords = [[tuple(coord) for coord in geometry['coordinates']]]
    else:
        for line in geometry['coordinates']:
            coords.append([tuple(coord) for coord in line])
            
    return {
        'type': 'MultiLineString',
        'coordinates': coords
    }

roads: dict = dict()
for road in data:
    _id = road['properties']['id']
    road['geometry'] = prepare_geometry(road['geometry'])
    
    if(_id in roads):
        roads[_id]['geometry']['coordinates'].extend(road['geometry']['coordinates'])
    else:
        roads[_id] = road

roads[173868646]['geometry']['coordinates']

[[(14.4070888, 50.124456),
  (14.40784007, 50.12440019),
  (14.40792462, 50.1244215)],
 [(14.40808668, 50.12340308), (14.4076876, 50.12366512)],
 [(14.4076876, 50.12366512),
  (14.40733297, 50.12389797),
  (14.40702799, 50.12400101),
  (14.40629802, 50.12403101),
  (14.40585973, 50.12400078),
  (14.40570397, 50.12398097),
  (14.40525696, 50.12366898),
  (14.40428831, 50.12338498),
  (14.40421716, 50.12358011),
  (14.40517701, 50.12418398),
  (14.40524987, 50.12423426),
  (14.40564396, 50.12442397),
  (14.40577305, 50.12444707),
  (14.40613309, 50.12452701),
  (14.4070888, 50.124456)],
 [(14.40792462, 50.1244215),
  (14.40821179, 50.12449389),
  (14.40854983, 50.1245952),
  (14.40876406, 50.12474902)],
 [(14.40827791, 50.12327751), (14.40808668, 50.12340308)],
 [(14.40876406, 50.12474902), (14.40883298, 50.1247985)]]

In [14]:
# Change MultiLineString geometries into LineString geometries

LineCoords = list[tuple[float, float]]
MultiCoords = list[LineCoords]

def construct_one_line(starting_line: LineCoords, coords: MultiCoords):
    """
    Concatenate all line parts into one line.
    """
    while(len(coords) > 0):
        end_point = starting_line[-1]
        found_line = None
        for line in reversed(coords):
            if(end_point == line[0]):
                # Found a line that connects to the end of starting line
                found_line = line
                break
        
        if(found_line is None):
            return None
        else:
            coords.remove(found_line)
            starting_line.remove(end_point)
            starting_line.extend(found_line)
    
    return starting_line

def find_starting_line(coords: MultiCoords) -> LineCoords:
    """
    Finds a line whose starting coordinate is not at the end of any other line.
    """
    for starting_line in reversed(coords):
        found_starting_line = True
        start_coord = starting_line[0]
        for line in reversed(coords):
            if start_coord == line[-1]:
                found_starting_line = False
                break
        
        if(found_starting_line == True):
            break
            
    if(found_starting_line == False):
        # Starting line not found
        return None
    
    coords.remove(starting_line)
    
    return starting_line

# Transform all roads into LineString
for road in roads.values():
    coords = road['geometry']['coordinates']
    starting_line: LineCoords = find_starting_line(coords)
    
    if(starting_line is None):
        print("Could not find starting line for {}".format(road['id']))
    
    line_string: LineCoords = construct_one_line(starting_line, coords)
    
    if(line_string is None):
        print("Could not form line string for {}".format(road['id']))
    
    road['geometry'] = {
        'type': 'LineString',
        'coordinates': line_string
    }

roads

{173868646: {'geometry': {'type': 'LineString',
   'coordinates': [(14.40827791, 50.12327751),
    (14.40808668, 50.12340308),
    (14.4076876, 50.12366512),
    (14.40733297, 50.12389797),
    (14.40702799, 50.12400101),
    (14.40629802, 50.12403101),
    (14.40585973, 50.12400078),
    (14.40570397, 50.12398097),
    (14.40525696, 50.12366898),
    (14.40428831, 50.12338498),
    (14.40421716, 50.12358011),
    (14.40517701, 50.12418398),
    (14.40524987, 50.12423426),
    (14.40564396, 50.12442397),
    (14.40577305, 50.12444707),
    (14.40613309, 50.12452701),
    (14.4070888, 50.124456),
    (14.40784007, 50.12440019),
    (14.40792462, 50.1244215),
    (14.40821179, 50.12449389),
    (14.40854983, 50.1245952),
    (14.40876406, 50.12474902),
    (14.40883298, 50.1247985)]},
  'type': 'Feature',
  'properties': {'source': 'openstreetmap.org',
   'kind': 'path',
   'kind_detail': 'path',
   'sort_rank': 354,
   'landuse_kind': 'grass',
   'min_zoom': 13.0,
   'id': 173868646}}}

In [15]:
printList(starting_line)

0. https://www.openstreetmap.org/?mlat=50.12327751&mlon=14.40827791&zoom=19
1. https://www.openstreetmap.org/?mlat=50.12340308&mlon=14.40808668&zoom=19
2. https://www.openstreetmap.org/?mlat=50.12366512&mlon=14.4076876&zoom=19
3. https://www.openstreetmap.org/?mlat=50.12389797&mlon=14.40733297&zoom=19
4. https://www.openstreetmap.org/?mlat=50.12400101&mlon=14.40702799&zoom=19
5. https://www.openstreetmap.org/?mlat=50.12403101&mlon=14.40629802&zoom=19
6. https://www.openstreetmap.org/?mlat=50.12400078&mlon=14.40585973&zoom=19
7. https://www.openstreetmap.org/?mlat=50.12398097&mlon=14.40570397&zoom=19
8. https://www.openstreetmap.org/?mlat=50.12366898&mlon=14.40525696&zoom=19
9. https://www.openstreetmap.org/?mlat=50.12338498&mlon=14.40428831&zoom=19
10. https://www.openstreetmap.org/?mlat=50.12358011&mlon=14.40421716&zoom=19
11. https://www.openstreetmap.org/?mlat=50.12418398&mlon=14.40517701&zoom=19
12. https://www.openstreetmap.org/?mlat=50.12423426&mlon=14.40524987&zoom=19
13. https:

# Test roads

In [16]:
from pymongo import MongoClient
import secrets as secrets
client = MongoClient(secrets.MONGODB_URI)
db = client["zoo_prague_db"]
coll = db['roads']
nodes_coll = db['road_nodes']

roads = list(coll.find({}))

In [17]:
unconnecteds = list()

for road in roads:
    connectors = [coord for coord in road['geometry']['coordinates'] if coord['is_connector'] == True]
    if(len(connectors) <= 1):
        unconnecteds.append(road)

for unconnected in unconnecteds:
    print(f"{unconnected['_id']},")

158092585,
469236907,
434925351,
354198717,
469714394,
469714393,
354198714,
354198713,
486592315,
486592316,
27126860,
27126880,
450545077,
469714387,
362893592,
469714388,
362893589,
363074839,
469714378,
486592321,
482471465,
552588356,
486592322,
30000155,
30000156,
114930688,
469918970,
469918971,
312719164,
362953296,
315259727,
315261382,
415223873,
376539609,
316332873,
27126873,
27126881,
101571181,
43273635,
43274561,
54045026,
33757185,
32397852,
43274560,
235051437,
222136535,
362893588,
362584472,
362953297,
362953298,
482466390,
482466388,
482467871,
517087279,
517087280,
27126691,
54042324,
54043877,
4818865,
4818869,
4818873,
32395595,
34179252,
255284380,
131712103,
291933344,
87453387,
25696834,
87453369,
32082233,
662082263,
186255665,
141180331,
180216048,
131712102,
374056499,
25582860,
89023051,
87453378,
53724551,
25400103,
31172077,
186255668,
404368549,
531085537,
662082365,
662082253,
337756271,
531085539,
548739729,
30025210,
104895213,
659327047,
173777403,


In [43]:
data2 = [
    { 
        "lon": 14.40461,
        "lat": 50.11837
    },
    {
        "lon": 14.40217,
        "lat": 50.11876
    },
            {
                "_id" : 871118945,
                "lon" : 14.40220503,
                "lat" : 50.11879346,
                "road_ids" : [ 
                    467896640
                ],
                "is_connector" : "false"
            }, 
            {
                "_id" : 746480045,
                "lon" : 14.40233052,
                "lat" : 50.11888021,
                "road_ids" : [ 
                    467896640, 
                    101571180
                ],
                "is_connector" : "true"
            }, 
            {
                "_id" : 151508718,
                "lon" : 14.40257657,
                "lat" : 50.11901528,
                "road_ids" : [ 
                    467896640
                ],
                "is_connector" : "false"
            }, 
            {
                "_id" : 295169626,
                "lon" : 14.40289377,
                "lat" : 50.11912432,
                "road_ids" : [ 
                    467896640, 
                    27126898
                ],
                "is_connector" : "true"
            }, 
            {
                "_id" : 933734346,
                "lon" : 14.40307617,
                "lat" : 50.1191586,
                "road_ids" : [ 
                    467896640
                ],
                "is_connector" : "false"
            }, 
            {
                "_id" : 254636480,
                "lon" : 14.40311718,
                "lat" : 50.11916631,
                "road_ids" : [ 
                    467896640
                ],
                "is_connector" : "false"
            }, 
            {
                "_id" : 969285398,
                "lon" : 14.40334041,
                "lat" : 50.1191583,
                "road_ids" : [ 
                    467896640
                ],
                "is_connector" : "false"
            }, 
            {
                "_id" : 594941651,
                "lon" : 14.40353965,
                "lat" : 50.1191181,
                "road_ids" : [ 
                    467896640
                ],
                "is_connector" : "false"
            }, 
            {
                "_id" : 941428227,
                "lon" : 14.40364081,
                "lat" : 50.11909206,
                "road_ids" : [ 
                    467896640
                ],
                "is_connector" : "false"
            }, 
            {
                "_id" : 154637248,
                "lon" : 14.40394929,
                "lat" : 50.11895186,
                "road_ids" : [ 
                    467896640
                ],
                "is_connector" : "false"
            }, 
            {
                "_id" : 678731707,
                "lon" : 14.40412131,
                "lat" : 50.11873229,
                "road_ids" : [ 
                    467896640
                ],
                "is_connector" : "false"
            }, 
            {
                "_id" : 664050358,
                "lon" : 14.40434652,
                "lat" : 50.11852891,
                "road_ids" : [ 
                    467896640, 
                    27126859
                ],
                "is_connector" : "true"
            }, 
            {
                "_id" : 360405165,
                "lon" : 14.40447992,
                "lat" : 50.11840507,
                "road_ids" : [ 
                    467896640
                ],
                "is_connector" : "false"
            }
]

data2 = [(node["lon"], node["lat"]) for node in data2]

In [44]:
printList(data2)

0. https://www.openstreetmap.org/?mlat=50.11837&mlon=14.40461&zoom=19
1. https://www.openstreetmap.org/?mlat=50.11876&mlon=14.40217&zoom=19
2. https://www.openstreetmap.org/?mlat=50.11852597&mlon=14.40197308&zoom=19
3. https://www.openstreetmap.org/?mlat=50.11867331&mlon=14.40207091&zoom=19
4. https://www.openstreetmap.org/?mlat=50.11879346&mlon=14.40220503&zoom=19
5. https://www.openstreetmap.org/?mlat=50.11888021&mlon=14.40233052&zoom=19
6. https://www.openstreetmap.org/?mlat=50.11901528&mlon=14.40257657&zoom=19
7. https://www.openstreetmap.org/?mlat=50.11912432&mlon=14.40289377&zoom=19
8. https://www.openstreetmap.org/?mlat=50.1191586&mlon=14.40307617&zoom=19
9. https://www.openstreetmap.org/?mlat=50.11916631&mlon=14.40311718&zoom=19
10. https://www.openstreetmap.org/?mlat=50.1191583&mlon=14.40334041&zoom=19
11. https://www.openstreetmap.org/?mlat=50.1191181&mlon=14.40353965&zoom=19
12. https://www.openstreetmap.org/?mlat=50.11909206&mlon=14.40364081&zoom=19
13. https://www.openstre

In [20]:
data3 = [784448524, 520984713, 893256765, 887511548, 590844987, 
         223488993, 843346703, 329154370, 336872439, 524226363, 
         269005178, 739676219, 669070785, 35708408, 190907276
        ]

data3 = [list(nodes_coll.find({"_id": _id}))[0] for _id in data3]
data3 = [(data["lon"], data["lat"]) for data in data3]

In [21]:
printList(data3)

0. https://www.openstreetmap.org/?mlat=50.11630238&mlon=14.4061781&zoom=19
1. https://www.openstreetmap.org/?mlat=50.11630117&mlon=14.40598002&zoom=19
2. https://www.openstreetmap.org/?mlat=50.11630031&mlon=14.40586926&zoom=19
3. https://www.openstreetmap.org/?mlat=50.116298&mlon=14.40560228&zoom=19
4. https://www.openstreetmap.org/?mlat=50.11633256&mlon=14.40547175&zoom=19
5. https://www.openstreetmap.org/?mlat=50.11636667&mlon=14.40547993&zoom=19
6. https://www.openstreetmap.org/?mlat=50.11644328&mlon=14.40547427&zoom=19
7. https://www.openstreetmap.org/?mlat=50.11649748&mlon=14.40546978&zoom=19
8. https://www.openstreetmap.org/?mlat=50.11656897&mlon=14.40545567&zoom=19
9. https://www.openstreetmap.org/?mlat=50.11661171&mlon=14.40541633&zoom=19
10. https://www.openstreetmap.org/?mlat=50.11664108&mlon=14.40540968&zoom=19
11. https://www.openstreetmap.org/?mlat=50.11671291&mlon=14.4053114&zoom=19
12. https://www.openstreetmap.org/?mlat=50.11670387&mlon=14.40485506&zoom=19
13. https://w

In [47]:
data4 = [
    (14.40460354753682, 50.11836232658393),(14.40474798, 50.11831239),
    (14.40447992, 50.11840507),(14.40434652, 50.11852891),(14.40412131, 50.11873229),
    (14.40394929, 50.11895186),(14.40364081, 50.11909206),(14.40353965, 50.1191181),
    (14.40334041, 50.1191583),(14.40311718, 50.11916631),(14.40307617, 50.1191586),
    (14.40289377, 50.11912432),(14.40257657, 50.11901528),(14.40233052, 50.11888021),
    (14.40220503, 50.11879346),(14.402168465383749, 50.11876070397075)
]

printList(data4)

0. https://www.openstreetmap.org/?mlat=50.11836232658393&mlon=14.40460354753682&zoom=19
1. https://www.openstreetmap.org/?mlat=50.11831239&mlon=14.40474798&zoom=19
2. https://www.openstreetmap.org/?mlat=50.11840507&mlon=14.40447992&zoom=19
3. https://www.openstreetmap.org/?mlat=50.11852891&mlon=14.40434652&zoom=19
4. https://www.openstreetmap.org/?mlat=50.11873229&mlon=14.40412131&zoom=19
5. https://www.openstreetmap.org/?mlat=50.11895186&mlon=14.40394929&zoom=19
6. https://www.openstreetmap.org/?mlat=50.11909206&mlon=14.40364081&zoom=19
7. https://www.openstreetmap.org/?mlat=50.1191181&mlon=14.40353965&zoom=19
8. https://www.openstreetmap.org/?mlat=50.1191583&mlon=14.40334041&zoom=19
9. https://www.openstreetmap.org/?mlat=50.11916631&mlon=14.40311718&zoom=19
10. https://www.openstreetmap.org/?mlat=50.1191586&mlon=14.40307617&zoom=19
11. https://www.openstreetmap.org/?mlat=50.11912432&mlon=14.40289377&zoom=19
12. https://www.openstreetmap.org/?mlat=50.11901528&mlon=14.40257657&zoom=19
